In [1]:
# importing necessary libraries

import requests
from lxml import html
from selenium import webdriver
import re
import time
from bs4 import BeautifulSoup

In [2]:
# extracting links of the poems

def get_hyperlinks_from_website(url):
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(2)
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        hyperlinks = [a['href'] for a in soup.find_all('a', href=True)]
        return hyperlinks
    except Exception as e:
        print("Error fetching website:", e)
        return []
    finally:
        driver.quit()

In [3]:
#extracting poems from url

def extract_text_from_xpath(url, xpaths):
    response = requests.get(url)
    if response.status_code == 200:
        tree = html.fromstring(response.content.decode('utf-8'))
        extracted_text = []
        for xpath in xpaths:
            elements = tree.xpath(xpath)
            if elements:
                extracted_text.extend([element.text_content() for element in elements])
        return extracted_text
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")
        return []

In [4]:
def save_to_txt(filename, text):
    text = text.replace('  ', '\n')
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    lines = [line for line in lines if len(line.split()) >= 5]
    lines = [line + '\n' if (idx + 1) % 2 == 0 else line for idx, line in enumerate(lines)]
    formatted_text = '\n'.join(lines)
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(formatted_text)

In [5]:
# pre-processing poems

def preprocess_text(text):
    
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    remove_empty_lines = r'^\s*$'
    text = re.sub(remove_empty_lines, '\n', text)
    
    special_characters = {
 'á': 'a', 'ă': 'a', 'ắ': 'a', 'ặ': 'a', 'ằ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ǎ': 'a', 'â': 'a', 'ấ': 'a', 'ậ': 'a', 'ầ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ä': 'a', 'ạ': 'a', 'à': 'a', 'ả': 'a', 'ā': 'a', 'ą': 'a', 'å': 'a', 'ǻ': 'a', 'ã': 'a', 'æ': 'a', 'ǽ': 'a', 'ɑ': 'a', 'ɐ': 'a', 'ɒ': 'a',
 'ḅ': 'b', 'ɓ': 'b', 'ß': 's',
 'ć': 'c', 'č': 'c', 'ç': 'c', 'ĉ': 'c', 'ɕ': 'c', 'ċ': 'c',
 'ď': 'd', 'ḓ': 'd', 'ḍ': 'd', 'ɗ': 'd', 'ḏ': 'd', 'đ': 'd', 'ɖ': 'd', 'ʤ': 'd', 'ǳ': 'd', 'ʣ': 'd', 'ʥ': 'd', 'ǆ': 'd', 'ð': 'd',
 'é': 'e', 'ĕ': 'e', 'ě': 'e', 'ê': 'e', 'ế': 'e', 'ệ': 'e', 'ề': 'e', 'ể': 'e', 'ễ': 'e', 'ë': 'e', 'ė': 'e', 'ẹ': 'e', 'è': 'e', 'ẻ': 'e', 'ē': 'e', 'ę': 'e', 'ẽ': 'e', 'ʒ': 'e', 'ǯ': 'e', 'ʓ': 'e', 'ɘ': 'e', 'ɜ': 'e', 'ɝ': 'e', 'ə': 'e', 'ɚ': 'e', 'ʚ': 'e', 'ɞ': 'e',
 'ƒ': 'f', 'ſ': 'f', 'ʩ': 'f', 'ﬁ': 'f', 'ﬂ': 'f', 'ʃ': 'f', 'ʆ': 'f', 'ʅ': 'f', 'ɟ': 'f', 'ʄ': 'f',
 'ǵ': 'g', 'ğ': 'g', 'ǧ': 'g', 'ģ': 'g', 'ĝ': 'g', 'ġ': 'g', 'ɠ': 'g', 'ḡ': 'g', 'ɡ': 'g', 'ɣ': 'g',
 'ḫ': 'h', 'ĥ': 'h', 'ḥ': 'h', 'ɦ': 'h', 'ẖ': 'h', 'ħ': 'h', 'ɧ': 'h', 'ɥ': 'h', 'ʮ': 'h', 'ʯ': 'h', 'ų': 'h',
 'í': 'i', 'ĭ': 'i', 'ǐ': 'i', 'î': 'i', 'ï': 'i', 'ị': 'i', 'ì': 'i', 'ỉ': 'i', 'ī': 'i', 'į': 'i', 'ɨ': 'i', 'ĩ': 'i', 'ɩ': 'i', 'ı': 'i',
 'ǰ': 'j', 'ĵ': 'j', 'ʝ': 'j','ȷ': 'j',
 'ķ': 'k', 'ḳ': 'k', 'ƙ': 'k', 'ḵ': 'k', 'ĸ': 'k', 'ʞ': 'k',
 'ĺ': 'l', 'ƚ': 'l', 'ɬ': 'l', 'ľ': 'l', 'ļ': 'l', 'ḽ': 'l', 'ḷ': 'l', 'ḹ': 'l', 'ḻ': 'l', 'ŀ': 'l', 'ɫ': 'l', 'ɭ': 'l', 'ł': 'l', 'ƛ': 'l', 'ɮ': 'l', 'ǉ': 'l', 'ʪ': 'l', 'ʫ': 'l',
 'ḿ': 'm', 'ṁ': 'm', 'ṃ': 'm', 'ɱ': 'm', 'ɯ': 'm', 'ɰ': 'm',
 'ŉ': 'n', 'ń': 'n', 'ň': 'n', 'ņ': 'n', 'ṋ': 'n', 'ṅ': 'n', 'ṇ': 'n', 'ǹ': 'n', 'ɲ': 'n', 'ṉ': 'n', 'ɳ': 'n', 'ñ': 'n', 'ǌ': 'n', 'ŋ': 'n', 'Ŋ': 'n',
 'ó': 'o', 'ŏ': 'o', 'ǒ': 'o', 'ô': 'o', 'ố': 'o', 'ộ': 'o', 'ồ': 'o', 'ổ': 'o', 'ỗ': 'o', 'ö': 'o', 'ọ': 'o', 'ő': 'o', 'ò': 'o', 'ỏ': 'o', 'ơ': 'o', 'ớ': 'o', 'ợ': 'o', 'ờ': 'o', 'ở': 'o', 'ỡ': 'o', 'ō': 'o', 'ǫ': 'o', 'ø': 'o', 'ǿ': 'o', 'õ': 'o', 'ɛ': 'o', 'ɔ': 'o', 'ɵ': 'o', 'ʘ': 'o', 'œ': 'o',
 'ɸ': 'p', 'þ': 'p', 'ʠ': 'p',
 'ŕ': 'r', 'ř': 'r', 'ŗ': 'r', 'ṙ': 'r', 'ṛ': 'r', 'ṝ': 'r', 'ɾ': 'r', 'ṟ': 'r', 'ɼ': 'r', 'ɽ': 'r', 'ɿ': 'r', 'ɹ': 'r', 'ɻ': 'r', 'ɺ': 'r',
 'ś': 's', 'š': 's', 'ş': 's', 'ŝ': 's', 'ș': 's', 'ṡ': 's', 'ṣ': 's', 'ʂ': 's',
 'ť': 't', 'ţ': 't', 'ṱ': 't', 'ț': 't', 'ẗ': 't', 'ṭ': 't', 'ṯ': 't', 'ʈ': 't', 'ŧ': 't', 'ʨ': 't', 'ʧ': 't', 'ʦ': 't', 'ʇ': 't',
 'ʉ': 'u', 'ú': 'u', 'ŭ': 'u', 'ǔ': 'u', 'û': 'u', 'ü': 'u', 'ǘ': 'u', 'ǚ': 'u', 'ǜ': 'u', 'ǖ': 'u', 'ụ': 'u', 'ű': 'u', 'ù': 'u', 'ủ': 'u', 'ư': 'u', 'ứ': 'u', 'ự': 'u', 'ừ': 'u', 'ử': 'u', 'ữ': 'u', 'ū': 'u', 'ů': 'u', 'ũ': 'u', 'ʊ': 'u',
 'ʋ': 'v', 'ʌ': 'v',
 'ẃ': 'w', 'ŵ': 'w', 'ẅ': 'w', 'ẁ': 'w', 'ʍ': 'w',
 'ý': 'y', 'ŷ': 'y', 'ÿ': 'y', 'ẏ': 'y', 'ỵ': 'y', 'ỳ': 'y', 'ƴ': 'y', 'ỷ': 'y', 'ȳ': 'y', 'ỹ': 'y', 'ʎ': 'y',
 'ź': 'z', 'ž': 'z', 'ʑ': 'z', 'ż': 'z', 'ẓ': 'z', 'ẕ': 'z', 'ʐ': 'z', 'ƶ': 'z'
    }
    
    for special, regular in special_characters.items():
        text = text.replace(special, regular)
    
    text = text.replace('.', '').replace('?', '')
    
    return text

In [6]:
#extracting artists

poets_xpaths = [f'//*[@id="content"]/div/div[2]/div[1]/div[3]/div[{i}]/div/div[2]/div[1]/a/h2' for i in range(1, 43)]

poets_links = 'https://www.rekhta.org/poets/top-read-poets'

poets = extract_text_from_xpath(poets_links, poets_xpaths)

poets = [i[:-1].replace(' ', '-').lower() for i in poets]

In [7]:
# saving links for poems

website_urls = [f'https://www.rekhta.org/poets/{poet}/ghazals' for poet in poets]
    
hyperlinks_list = [get_hyperlinks_from_website(website_url) for website_url in website_urls]

all_hyperlinks = [link for sublist in hyperlinks_list for link in sublist]

poem_links = list(set([link for link in all_hyperlinks if link.startswith('https://www.rekhta.org/ghazals/')]))

In [8]:
# executing scraping

if __name__ == "__main__":

    xpaths = []
    for i in range(1, 16):
        for j in [1, 2]:
            xpath = f'//*[@id="content"]/div/div/div[2]/div[1]/div[{i}]/div/p[{j}]'
            xpaths.append(xpath)

    for idx, link in enumerate(poem_links):
        
        extracted_text = extract_text_from_xpath(link, xpaths)
        
        extracted_text = ' '.join(extracted_text)
        
        extracted_text = preprocess_text(extracted_text)

        if extracted_text:
            output_file = f"ghazal_{idx + 1}.txt"
            save_to_txt(output_file, extracted_text)
            print(f"Extraction completed for link {idx + 1}. Text saved to {output_file}.")
        else:
            print(f"No text was extracted for link {idx + 1}.")

Extraction completed for link 1. Text saved to ghazal_1.txt.
Extraction completed for link 2. Text saved to ghazal_2.txt.
Extraction completed for link 3. Text saved to ghazal_3.txt.
Extraction completed for link 4. Text saved to ghazal_4.txt.
Extraction completed for link 5. Text saved to ghazal_5.txt.
Extraction completed for link 6. Text saved to ghazal_6.txt.
Extraction completed for link 7. Text saved to ghazal_7.txt.
Extraction completed for link 8. Text saved to ghazal_8.txt.
Extraction completed for link 9. Text saved to ghazal_9.txt.
Extraction completed for link 10. Text saved to ghazal_10.txt.
Extraction completed for link 11. Text saved to ghazal_11.txt.
Extraction completed for link 12. Text saved to ghazal_12.txt.
Extraction completed for link 13. Text saved to ghazal_13.txt.
Extraction completed for link 14. Text saved to ghazal_14.txt.
Extraction completed for link 15. Text saved to ghazal_15.txt.
Extraction completed for link 16. Text saved to ghazal_16.txt.
Extraction

Extraction completed for link 131. Text saved to ghazal_131.txt.
Extraction completed for link 132. Text saved to ghazal_132.txt.
Extraction completed for link 133. Text saved to ghazal_133.txt.
Extraction completed for link 134. Text saved to ghazal_134.txt.
Extraction completed for link 135. Text saved to ghazal_135.txt.
Extraction completed for link 136. Text saved to ghazal_136.txt.
Extraction completed for link 137. Text saved to ghazal_137.txt.
Extraction completed for link 138. Text saved to ghazal_138.txt.
Extraction completed for link 139. Text saved to ghazal_139.txt.
Extraction completed for link 140. Text saved to ghazal_140.txt.
Extraction completed for link 141. Text saved to ghazal_141.txt.
Extraction completed for link 142. Text saved to ghazal_142.txt.
Extraction completed for link 143. Text saved to ghazal_143.txt.
Extraction completed for link 144. Text saved to ghazal_144.txt.
Extraction completed for link 145. Text saved to ghazal_145.txt.
Extraction completed for 

Extraction completed for link 258. Text saved to ghazal_258.txt.
Extraction completed for link 259. Text saved to ghazal_259.txt.
Extraction completed for link 260. Text saved to ghazal_260.txt.
Extraction completed for link 261. Text saved to ghazal_261.txt.
Extraction completed for link 262. Text saved to ghazal_262.txt.
Extraction completed for link 263. Text saved to ghazal_263.txt.
Extraction completed for link 264. Text saved to ghazal_264.txt.
Extraction completed for link 265. Text saved to ghazal_265.txt.
Extraction completed for link 266. Text saved to ghazal_266.txt.
Extraction completed for link 267. Text saved to ghazal_267.txt.
Extraction completed for link 268. Text saved to ghazal_268.txt.
Extraction completed for link 269. Text saved to ghazal_269.txt.
Extraction completed for link 270. Text saved to ghazal_270.txt.
Extraction completed for link 271. Text saved to ghazal_271.txt.
Extraction completed for link 272. Text saved to ghazal_272.txt.
Extraction completed for 

Extraction completed for link 385. Text saved to ghazal_385.txt.
Extraction completed for link 386. Text saved to ghazal_386.txt.
Extraction completed for link 387. Text saved to ghazal_387.txt.
Extraction completed for link 388. Text saved to ghazal_388.txt.
Extraction completed for link 389. Text saved to ghazal_389.txt.
Extraction completed for link 390. Text saved to ghazal_390.txt.
Extraction completed for link 391. Text saved to ghazal_391.txt.
Extraction completed for link 392. Text saved to ghazal_392.txt.
Extraction completed for link 393. Text saved to ghazal_393.txt.
Extraction completed for link 394. Text saved to ghazal_394.txt.
Extraction completed for link 395. Text saved to ghazal_395.txt.
Extraction completed for link 396. Text saved to ghazal_396.txt.
Extraction completed for link 397. Text saved to ghazal_397.txt.
Extraction completed for link 398. Text saved to ghazal_398.txt.
Extraction completed for link 399. Text saved to ghazal_399.txt.
Extraction completed for 

Extraction completed for link 512. Text saved to ghazal_512.txt.
Extraction completed for link 513. Text saved to ghazal_513.txt.
Extraction completed for link 514. Text saved to ghazal_514.txt.
Extraction completed for link 515. Text saved to ghazal_515.txt.
Extraction completed for link 516. Text saved to ghazal_516.txt.
Extraction completed for link 517. Text saved to ghazal_517.txt.
Extraction completed for link 518. Text saved to ghazal_518.txt.
Extraction completed for link 519. Text saved to ghazal_519.txt.
Extraction completed for link 520. Text saved to ghazal_520.txt.
Extraction completed for link 521. Text saved to ghazal_521.txt.
Extraction completed for link 522. Text saved to ghazal_522.txt.
Extraction completed for link 523. Text saved to ghazal_523.txt.
Extraction completed for link 524. Text saved to ghazal_524.txt.
Extraction completed for link 525. Text saved to ghazal_525.txt.
Extraction completed for link 526. Text saved to ghazal_526.txt.
Extraction completed for 

...

Extraction completed for link 2742. Text saved to ghazal_2742.txt.
Extraction completed for link 2743. Text saved to ghazal_2743.txt.
Extraction completed for link 2744. Text saved to ghazal_2744.txt.
Extraction completed for link 2745. Text saved to ghazal_2745.txt.
Extraction completed for link 2746. Text saved to ghazal_2746.txt.
Extraction completed for link 2747. Text saved to ghazal_2747.txt.
Extraction completed for link 2748. Text saved to ghazal_2748.txt.
Extraction completed for link 2749. Text saved to ghazal_2749.txt.
Extraction completed for link 2750. Text saved to ghazal_2750.txt.
Extraction completed for link 2751. Text saved to ghazal_2751.txt.
Extraction completed for link 2752. Text saved to ghazal_2752.txt.
Extraction completed for link 2753. Text saved to ghazal_2753.txt.
Extraction completed for link 2754. Text saved to ghazal_2754.txt.
Extraction completed for link 2755. Text saved to ghazal_2755.txt.
Extraction completed for link 2756. Text saved to ghazal_2756.

Extraction completed for link 2865. Text saved to ghazal_2865.txt.
Extraction completed for link 2866. Text saved to ghazal_2866.txt.
Extraction completed for link 2867. Text saved to ghazal_2867.txt.
Extraction completed for link 2868. Text saved to ghazal_2868.txt.
Extraction completed for link 2869. Text saved to ghazal_2869.txt.
Extraction completed for link 2870. Text saved to ghazal_2870.txt.
Extraction completed for link 2871. Text saved to ghazal_2871.txt.
Extraction completed for link 2872. Text saved to ghazal_2872.txt.
Extraction completed for link 2873. Text saved to ghazal_2873.txt.
Extraction completed for link 2874. Text saved to ghazal_2874.txt.
Extraction completed for link 2875. Text saved to ghazal_2875.txt.
Extraction completed for link 2876. Text saved to ghazal_2876.txt.
Extraction completed for link 2877. Text saved to ghazal_2877.txt.
Extraction completed for link 2878. Text saved to ghazal_2878.txt.
Extraction completed for link 2879. Text saved to ghazal_2879.

Extraction completed for link 2988. Text saved to ghazal_2988.txt.
Extraction completed for link 2989. Text saved to ghazal_2989.txt.
Extraction completed for link 2990. Text saved to ghazal_2990.txt.
Extraction completed for link 2991. Text saved to ghazal_2991.txt.
Extraction completed for link 2992. Text saved to ghazal_2992.txt.
Extraction completed for link 2993. Text saved to ghazal_2993.txt.
Extraction completed for link 2994. Text saved to ghazal_2994.txt.
Extraction completed for link 2995. Text saved to ghazal_2995.txt.
Extraction completed for link 2996. Text saved to ghazal_2996.txt.
Extraction completed for link 2997. Text saved to ghazal_2997.txt.
Extraction completed for link 2998. Text saved to ghazal_2998.txt.
Extraction completed for link 2999. Text saved to ghazal_2999.txt.
Extraction completed for link 3000. Text saved to ghazal_3000.txt.
Extraction completed for link 3001. Text saved to ghazal_3001.txt.
Extraction completed for link 3002. Text saved to ghazal_3002.

Extraction completed for link 3111. Text saved to ghazal_3111.txt.
Extraction completed for link 3112. Text saved to ghazal_3112.txt.
Extraction completed for link 3113. Text saved to ghazal_3113.txt.
Extraction completed for link 3114. Text saved to ghazal_3114.txt.
Extraction completed for link 3115. Text saved to ghazal_3115.txt.
Extraction completed for link 3116. Text saved to ghazal_3116.txt.
Extraction completed for link 3117. Text saved to ghazal_3117.txt.
Extraction completed for link 3118. Text saved to ghazal_3118.txt.
Extraction completed for link 3119. Text saved to ghazal_3119.txt.
Extraction completed for link 3120. Text saved to ghazal_3120.txt.
Extraction completed for link 3121. Text saved to ghazal_3121.txt.
Extraction completed for link 3122. Text saved to ghazal_3122.txt.
Extraction completed for link 3123. Text saved to ghazal_3123.txt.
Extraction completed for link 3124. Text saved to ghazal_3124.txt.
Extraction completed for link 3125. Text saved to ghazal_3125.

Extraction completed for link 3234. Text saved to ghazal_3234.txt.
Extraction completed for link 3235. Text saved to ghazal_3235.txt.
Extraction completed for link 3236. Text saved to ghazal_3236.txt.
Extraction completed for link 3237. Text saved to ghazal_3237.txt.
Extraction completed for link 3238. Text saved to ghazal_3238.txt.
Extraction completed for link 3239. Text saved to ghazal_3239.txt.
Extraction completed for link 3240. Text saved to ghazal_3240.txt.
Extraction completed for link 3241. Text saved to ghazal_3241.txt.
Extraction completed for link 3242. Text saved to ghazal_3242.txt.
Extraction completed for link 3243. Text saved to ghazal_3243.txt.
Extraction completed for link 3244. Text saved to ghazal_3244.txt.
Extraction completed for link 3245. Text saved to ghazal_3245.txt.
Extraction completed for link 3246. Text saved to ghazal_3246.txt.
Extraction completed for link 3247. Text saved to ghazal_3247.txt.
Extraction completed for link 3248. Text saved to ghazal_3248.

In [9]:
xx

NameError: name 'xx' is not defined